In [1]:
import pandas as pd
import numpy as np
import gc
import re

加载数据

In [2]:
train_data_origin = pd.read_csv('train_stage2_update_20200320.csv') #读入原始的训练数据的特征
train_label_origin = pd.read_csv('train_label.csv') #读入原始的训练数据的标签
train_data = train_data_origin.merge(train_label_origin, on='ID', how='left') #将训练数据的特征和标签连接起来，左连接，特征为基准

test_data_origin = pd.read_csv('test_stage2_update_20200320.csv') #读入原始的测试数据的特征
test_data_origin['Label'] = -1 #添加Label列
test_data = test_data_origin

del train_data_origin, test_data_origin, train_label_origin
gc.collect()

D:\anaconda3-4.3.1\envs\python36\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0

检查数据

In [3]:
train_data.shape

(14366, 665)

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14366 entries, 0 to 14365
Columns: 665 entries, ID to Label
dtypes: float64(656), int64(2), object(7)
memory usage: 73.0+ MB


In [5]:
columns = list(train_data.columns)
columns

['ID',
 '企业类型',
 '经营期限至',
 '登记机关',
 '企业状态',
 '邮政编码',
 '投资总额',
 '注册资本',
 '核准日期',
 '行业代码',
 '注销时间',
 '经营期限自',
 '成立日期',
 '行业门类',
 '企业类别',
 '管辖机关',
 '经营范围',
 '增值税',
 '企业所得税',
 '印花税',
 '教育费',
 '城建税',
 '递延收益_年初数',
 '长期负债合计_年初数',
 '长期借款_年初数',
 '长期应付款_年初数',
 '短期借款_年初数',
 '递延所得税负债_年初数',
 '非流动负债合计_年初数',
 '负债合计_年初数',
 '负债和所有者权益总计_年初数',
 '交易性金融负债_年初数',
 '流动负债合计_年初数',
 '其他非流动负债_年初数',
 '其他负债（或长期负债）_年初数',
 '其他流动负债_年初数',
 '其他应付款_年初数',
 '其他应交款_年初数',
 '实收资本（股本）_年初数',
 '实收资本（或股本）净额_年初数',
 '所有者权益合计_年初数',
 '未分配利润_年初数',
 '小企业_应付利润_年初数',
 '应付福利费_年初数',
 '应付股利_年初数',
 '应付利息_年初数',
 '应付票据_年初数',
 '应付职工薪酬_年初数',
 '应付债券_年初数',
 '应付账款_年初数',
 '预计负债_年初数',
 '应交税费_年初数',
 '一年内到期的非流动负债_年初数',
 '预提费用_年初数',
 '预收款项_年初数',
 '盈余公积_年初数',
 '资本公积_年初数',
 '专项应付款_年初数',
 '递延收益_年末数',
 '长期负债合计_年末数',
 '长期借款_年末数',
 '长期应付款_年末数',
 '短期借款_年末数',
 '递延所得税负债_年末数',
 '非流动负债合计_年末数',
 '负债合计_年末数',
 '负债和所有者权益总计_年末数',
 '交易性金融负债_年末数',
 '流动负债合计_年末数',
 '其他非流动负债_年末数',
 '其他负债（或长期负债）_年末数',
 '其他流动负债_年末数',
 '其他应付款_年末数',
 '其他应交款_年末数',
 '实收资本（股本）_年末数',
 '实收资本（或股本）净额_

In [3]:
#计算缺失率
def get_data_null_rate(train, test):
    train_null_rate = (train.isnull().sum() / len(train)).reset_index().rename(columns={0:'train'})
    test_null_rate = (test.isnull().sum() / len(test)).reset_index().rename(columns={0:'test'})
    data_null_rate = pd.merge(train_null_rate, test_null_rate, on='index', how='inner')

    return data_null_rate
data_null_rate = get_data_null_rate(train_data.copy(), test_data.copy())
data_null_rate.head()

,index,train,test
0,ID,0.000000,0.000000
1,企业类型,0.053460,0.051577
2,经营期限至,0.804956,0.797452
3,登记机关,0.053460,0.051577
4,企业状态,0.053460,0.051577


In [4]:
#删除训练集和验证集中缺失率为1 和 每列不同的值的个数小于2 的特征
remove_cols = list(data_null_rate[(data_null_rate['train']==1) | (data_null_rate['test']==1)]['index'])
for col in train_data.columns:
    if train_data[col].nunique() < 2:
        remove_cols.append(col)
remove_cols
train_data = train_data[[col for col in train_data.columns if col not in remove_cols]]
test_data = test_data[[col for col in test_data.columns if col not in remove_cols]]
all_data = train_data.append(test_data)

In [5]:
print('{}\n{}\n{}'.format(all_data.shape, train_data.shape, test_data.shape))

(23944, 288)
(14366, 288)
(9578, 288)


数据预处理

In [6]:
#处理时间
date_cols = ['经营期限自','经营期限至','成立日期','核准日期','注销时间']
all_data['date_null_rate'] = all_data[date_cols].apply(lambda x: x.isnull().sum(), axis=1)
all_data[date_cols].dtypes
# 分割出日期类型中的小时
def split_date(x):
    if type(x) == float:
        return np.nan
    else:
        return int(x.split(':')[0])
for col in date_cols:
    all_data[col] = all_data[col].apply(lambda x: split_date(x))


In [7]:
# 处理邮编
def youbian_encode(x):
    if len(re.match('[\d]*',str(x)).group())==0:
        x = 999999
    elif len(str(int(float(x))))!=6:
        if len(str(int(float(x))))>3:
            x = int(str(int(float(x)))+'0'*(6-len(str(int(float(x))))))
        elif len(str(int(float(x))))==3:
            x=int('266'+str(int(float(x))))
        elif len(str(int(float(x))))==2:
            x = int('2667'+str(int(float(x))))
        else:
            x = 266000
    elif pd.isna(x):
        x=np.nan
    else:
        x = int(float(x))
    return x
all_data['邮政编码'].fillna(999999,inplace=True)
all_data['邮政编码'] = all_data['邮政编码'].apply(lambda x: youbian_encode(x))
all_data['邮政编码']

0       266300
1       266000
2       266000
3       266000
4       266400
         ...  
9573    266000
9574    266000
9575    266043
9576    266000
9577    999999
Name: 邮政编码, Length: 23944, dtype: int64

In [8]:
#处理离群值
def outliers(data, col_name, scale=3):
    data_col = data[col_name]
    q1 = data_col.quantile(0.25)
    q3 = data_col.quantile(0.75)
    IQR = q3 - q1
    
    data_col[data_col < q1 - (scale * IQR)] = q1 - (scale * IQR)
    data_col[data_col > q3 + (scale * IQR)] = q3 + (scale * IQR)
    
    return data[col_name]

for col in ['注册资本','增值税','企业所得税','印花税','教育费','城建税','年度参保总额']:
    all_data[col] = outliers(all_data, col, 10)
    
all_data

D:\anaconda3-4.3.1\envs\python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda3-4.3.1\envs\python36\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,ID,企业类型,经营期限至,登记机关,企业状态,邮政编码,投资总额,注册资本,核准日期,行业代码,...,处罚程度_1,处罚类型_1,罚款金额_sum,罚款金额_count,罚款金额_max,罚款金额_min,最新参保人数,年度参保总额,Label,date_null_rate
0,1,9.0,NaN,1.0,0.0,266300,19.110000,19.110000,0.0,5154.0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.088573,26.491406,0,2
1,3,11.0,NaN,5.0,0.0,266000,29.154194,19.210446,0.0,7214.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6.328575,32.574067,0,2
2,9,0.0,NaN,3.0,0.0,266000,NaN,19.120045,13.0,2110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,8.988578,40.046879,0,2
3,10,0.0,42.0,9.0,0.0,266000,NaN,19.160223,0.0,5163.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6.468575,33.026395,0,1
4,14,0.0,NaN,2.0,0.0,266400,NaN,19.210446,0.0,5439.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6.188575,32.189379,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,23932,7.0,27.0,2.0,1.0,266000,132.609397,19.762898,0.0,3491.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,0
9574,23935,0.0,NaN,3.0,0.0,266000,NaN,19.135111,0.0,1329.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,2
9575,23936,2.0,NaN,0.0,0.0,266043,NaN,19.361115,0.0,8060.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,2
9576,23938,0.0,NaN,3.0,0.0,266000,NaN,24.433632,0.0,3140.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,2


In [9]:
#经营范围
def process_jingyingfanwei(x):
    vals = [i.strip() for i in x[1:-1].split(',')]
    return ' '.join(vals)
all_data['经营范围'] = all_data['经营范围'].apply(lambda x: process_jingyingfanwei(x))

特征工程

In [10]:
#利用tfidf和truncatedSVD对经营范围做主题分析
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
def tf_idf(all_data, col, n_components=5):
    Tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer = Tfidf_vectorizer.fit_transform(all_data[col])
    svd = TruncatedSVD(n_components=n_components, n_iter=20, random_state=1024)
    vectorizer_svd = svd.fit_transform(tfidf_vectorizer)
    vectorizer_svd = pd.DataFrame(vectorizer_svd)
    vectorizer_svd.columns = ['SVD_{}_{}'.format(col, i) for i in range(n_components)]
    vectorizer_svd = vectorizer_svd.reset_index(drop = True)
    return vectorizer_svd
#经营范围
jingying_svd = tf_idf(all_data, '经营范围', n_components=5)
all_data =  all_data.reset_index(drop=True)
all_data = pd.concat([all_data, jingying_svd], axis=1)

In [11]:
#资产
money_cols = ['投资总额','注册资本','增值税','企业所得税','印花税','教育费','城建税','年度参保总额']
all_data['教育费/增值税'] = all_data['教育费']/all_data['增值税']
all_data['教育费/企业所得'] = all_data['教育费']/all_data['企业所得税']
all_data['企业所得/教育费'] = all_data['企业所得税']/all_data['教育费']
all_data['企业所得税/城建税'] = all_data['企业所得税']/all_data['城建税']
all_data['城建税/教育费'] = all_data['城建税']/all_data['教育费']

In [12]:
#案件
# anjian_cols = [col for col in all_data.columns if col.find('案件')!=-1]
all_data['1_3_anjian'] = all_data[['1月案件数','2月案件数','3月案件数']].apply(lambda x: x.sum(),axis=1)
all_data['1_3_anjian'].replace(0,np.nan,inplace=True)
all_data['4_5_anjian'] = all_data[['4月案件数','5月案件数','6月案件数']].apply(lambda x: x.sum(),axis=1)
all_data['4_5_anjian'].replace(0,np.nan,inplace=True)
all_data['7_9_anjian'] = all_data[['7月案件数','8月案件数','9月案件数']].apply(lambda x: x.sum(),axis=1)
all_data['7_9_anjian'].replace(0,np.nan,inplace=True)
all_data['10_12_anjian'] = all_data[['10月案件数','11月案件数','12月案件数']].apply(lambda x: x.sum(),axis=1)
all_data['10_12_anjian'].replace(0,np.nan,inplace=True)

In [13]:
zichan_cols = [col for col in all_data.columns if col.find('变更')!=-1]
all_data['zichan'] = (all_data['资本变更后']-all_data['资本变更前'])

In [14]:
#处理行业门类
def process_hangye(x):
    if x>12:
        return 12 
#     elif x==4:
#         return 3
all_data['行业门类'] = all_data['行业门类'].apply(lambda x: process_hangye(x))

In [15]:
def process_rank(x):
    res = [str(int(i)) for i in list(x)]
    return  ' '.join(res)
cols2=[]
tmp = pd.DataFrame()
for col1 in ['投资总额','注册资本','增值税','企业所得税','印花税','教育费','城建税']:
    for col2 in ['邮政编码','行业代码','企业类型','行业门类']:
        tmp[col1+'_'+col2] = all_data[col1].groupby(all_data[col2]).rank(ascending=0,method='max')
tmp.fillna(-1,inplace=True)
tmp['rank'] = tmp.apply(lambda x:process_rank(x),axis=1)
# rank 编码
rank_svd = tf_idf(tmp, 'rank', n_components=3)
all_data = all_data.reset_index(drop= True)
all_data = pd.concat([all_data, rank_svd], axis=1)

构建模型

In [16]:
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple lightgbm 
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple xgboost
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple catboost
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
# from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
print('ok')

ok


In [17]:
def train_lgb_model(train_, valid_, id_name, label_name, categorical_feature=None, seed=1024, is_shuffle=True):
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    train_['res'] = 0
    pred = [col for col in train_.columns if col not in [id_name,label_name,'res']]
    sub_preds = np.zeros((valid_.shape[0], folds.n_splits))
    params = {
        'learning_rate': 0.01,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': ['binary_logloss', 'auc'],
        'num_leaves':32,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'seed': 1,
        # 'device': 'gpu',
        'bagging_seed': 1,
        'feature_fraction_seed':7,
        'min_data_in_leaf': 28,
        'nthread': -1,
        'verbose': -1, 
    }
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_, train_[label_name]), start=1):
        print(f'the {n_fold} training start ...')
        train_x, train_y = train_[pred].iloc[train_idx], train_[label_name].iloc[train_idx]
        valid_x, valid_y = train_[pred].iloc[valid_idx], train_[label_name].iloc[valid_idx]
        dtrain = lgb.Dataset(train_x, label=train_y)
        dvalid = lgb.Dataset(valid_x, label=valid_y)
        clf = lgb.train(
            params=params,
            train_set=dtrain,
            num_boost_round=1000,
            valid_sets=[dtrain, dvalid],
            early_stopping_rounds=100,
            verbose_eval=100,
        )
        train_pred = clf.predict(valid_x, num_iteration=clf.best_iteration)
        train_['res'].iloc[valid_idx] = train_['res'].iloc[valid_idx] + train_pred
        
        # 在验证集上
        sub_preds[:, n_fold - 1] = clf.predict(valid_[pred], num_iteration=clf.best_iteration)
    valid_[label_name] = np.mean(sub_preds, axis=1)

    return train_[[id_name, 'res']],valid_[[id_name,label_name]]

In [21]:
def train_xgb_model(train_, valid_, id_name, label_name, categorical_feature=None, seed=1024, is_shuffle=True):
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    train_['res'] = 0
    pred = [col for col in train_.columns if col not in [id_name,label_name,'res']]
    valid_dms = xgb.DMatrix(valid_[pred])
    sub_preds = np.zeros((valid_.shape[0], folds.n_splits))
    print('sub_preds{}'.format(sub_preds.shape))
    params = {'eta': 0.1,
              'tree_method': "hist",
              'grow_policy': "lossguide",
              'max_leaves': 1024,
              'max_depth': 7,
              'subsample': 0.9,
              'colsample_bytree': 0.7,
              'colsample_bylevel': 0.7,
              'alpha': 4,
              'objective': 'binary:logistic',
              'scale_pos_weight': 9,
              'eval_metric': ['auc'],
              'nthread': 8,
              'random_state': 99,
              'silent': True}
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_, train_[label_name]), start=1):
        train_x, train_y = train_[pred].iloc[train_idx], train_[label_name].iloc[train_idx]
        valid_x, valid_y = train_[pred].iloc[valid_idx], train_[label_name].iloc[valid_idx]
        dtrain = xgb.DMatrix(train_x, label=train_y)
        dvalid = xgb.DMatrix(valid_x, label=valid_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
        clf = xgb.train(params, dtrain, 2000, watchlist,
                          maximize=True, early_stopping_rounds=50, verbose_eval=10)
        train_pred = clf.predict(dvalid)
        train_['res'].iloc[valid_idx] = train_['res'].iloc[valid_idx] + train_pred
        sub_preds[:, n_fold - 1] = clf.predict(valid_dms)
    valid_[label_name] = np.mean(sub_preds, axis=1)

    return train_[[id_name, 'res']],valid_[[id_name,label_name]]

In [18]:
cols = ['城建税/教育费', '印花税', '年度参保总额', '最新参保人数', '企业所得税', '行业代码', '增值税', '教育费/增值税', 'SVD_经营范围_4', 'SVD_经营范围_0', '城建税', 'SVD_经营范围_3', '登记注册类型代码', '注册资本', '经营期限自', 'SVD_经营范围_1', '成立日期', 'SVD_经营范围_2', '企业类型', '企业所得税/城建税', '教育费', '教育费/企业所得', '邮政编码', '登记机关', '核准日期', '经营期限至', '管辖机关', '诉讼总数量', '企业所得/教育费', '货币资金_年初数', '投资总额', '货币资金_年末数', '纳税人状态代码', '7_9_anjian', 'zichan', '1_3_anjian', '其他应收款_年末数', '其他应收款_年初数', '应收账款_年初数', '未分配利润_年初数', '预付款项_年末数', '预付款项_年初数', 'date_null_rate', '4_5_anjian', '所有者权益合计_年初数', '企业状态', '应付账款_年初数', '实收资本（股本）_年初数']
train_ = all_data[all_data['Label']!=-1]
valid_ = all_data[all_data['Label']==-1]

In [19]:
print('lgb模型')
lgb_train_pred, lgb_test_pred = train_lgb_model(train_[cols+['ID','Label']], valid_[cols+['ID']], 'ID', 'Label',seed=1024)

lgb模型
the 1 training start ...


D:\anaconda3-4.3.1\envs\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.136477	training's auc: 0.973528	valid_1's binary_logloss: 0.155529	valid_1's auc: 0.928791
[200]	training's binary_logloss: 0.110832	training's auc: 0.981522	valid_1's binary_logloss: 0.142	valid_1's auc: 0.929775
[300]	training's binary_logloss: 0.0964611	training's auc: 0.988311	valid_1's binary_logloss: 0.138865	valid_1's auc: 0.930062
Early stopping, best iteration is:
[218]	training's binary_logloss: 0.107703	training's auc: 0.982976	valid_1's binary_logloss: 0.140803	valid_1's auc: 0.930584


D:\anaconda3-4.3.1\envs\python36\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\anaconda3-4.3.1\envs\python36\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


the 2 training start ...
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.135783	training's auc: 0.971372	valid_1's binary_logloss: 0.156536	valid_1's auc: 0.918196
Early stopping, best iteration is:
[48]	training's binary_logloss: 0.165489	training's auc: 0.961607	valid_1's binary_logloss: 0.178182	valid_1's auc: 0.92144
the 3 training start ...
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.137085	training's auc: 0.971829	valid_1's binary_logloss: 0.153437	valid_1's auc: 0.937892
Early stopping, best iteration is:
[97]	training's binary_logloss: 0.138263	training's auc: 0.971589	valid_1's binary_logloss: 0.154251	valid_1's auc: 0.937999
the 4 training start ...
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.134223	training's auc: 0.974626	valid_1's binary_logloss: 0.162614	valid_1's auc: 0.902934
[200]	training's binary_logloss: 0.1086

D:\anaconda3-4.3.1\envs\python36\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
print('xgb模型')
xgb_train_pred, xgb_test_pred = train_xgb_model(train_[cols+['ID','Label']], valid_[cols+['ID','Label']], 'ID', 'Label',seed=1024)

xgb模型
sub_preds(9578, 5)


D:\anaconda3-4.3.1\envs\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


[0]	train-auc:0.86482	valid-auc:0.84167
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[10]	train-auc:0.89402	valid-auc:0.86966
[20]	train-auc:0.92197	valid-auc:0.88264
[30]	train-auc:0.92365	valid-auc:0.86320
[40]	train-auc:0.87398	valid-auc:0.83744
[50]	train-auc:0.91558	valid-auc:0.87394
Stopping. Best iteration:
[2]	train-auc:0.91804	valid-auc:0.91459



D:\anaconda3-4.3.1\envs\python36\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\anaconda3-4.3.1\envs\python36\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[0]	train-auc:0.88021	valid-auc:0.89144
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[10]	train-auc:0.78564	valid-auc:0.77880
[20]	train-auc:0.88377	valid-auc:0.83934
[30]	train-auc:0.91102	valid-auc:0.85610
[40]	train-auc:0.88463	valid-auc:0.83157
[50]	train-auc:0.93249	valid-auc:0.87343
Stopping. Best iteration:
[2]	train-auc:0.92348	valid-auc:0.90886

[0]	train-auc:0.92051	valid-auc:0.92241
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[10]	train-auc:0.87277	valid-auc:0.84090
[20]	train-auc:0.89581	valid-auc:0.88194
[30]	train-auc:0.89454	valid-auc:0.84784
[40]	train-auc:0.75093	valid-auc:0.73797
[50]	train-auc:0.86488	valid-auc:0.81792
Stopping. Best iteration:
[0]	train-auc:0.92051	valid-auc:0.92241

[0]	train-auc:0.84615	valid-auc:0.79420
Multiple eval metrics have been passed: 'valid-auc

D:\anaconda3-4.3.1\envs\python36\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
print(xgb_train_pred)
print(xgb_test_pred)

          ID           res
0          1  7.245567e-20
1          3  1.434011e-19
2          9  1.018917e-05
3         10  8.316805e-01
4         14  5.041163e-13
...      ...           ...
14361  23937  4.403580e-08
14362  23940  5.137439e-04
14363  23941  7.446313e-01
14364  23942  1.313978e-16
14365  23943  1.545222e-03

[14366 rows x 2 columns]
          ID     Label
14366      0  0.000147
14367      2  0.000044
14368      4  0.000010
14369      5  0.020284
14370      6  0.004086
...      ...       ...
23939  23932  0.202430
23940  23935  0.457755
23941  23936  0.000196
23942  23938  0.000390
23943  23939  0.799009

[9578 rows x 2 columns]


In [24]:
print(lgb_train_pred)
print(lgb_test_pred)

          ID       res
0          1  0.027486
1          3  0.003725
2          9  0.152454
3         10  0.161165
4         14  0.016908
...      ...       ...
14361  23937  0.009489
14362  23940  0.009487
14363  23941  0.180452
14364  23942  0.003755
14365  23943  0.027483

[14366 rows x 2 columns]
          ID     Label
14366      0  0.020067
14367      2  0.026425
14368      4  0.020231
14369      5  0.021217
14370      6  0.022491
...      ...       ...
23939  23932  0.021508
23940  23935  0.329042
23941  23936  0.020125
23942  23938  0.020646
23943  23939  0.383180

[9578 rows x 2 columns]


In [23]:
lgb_train_pred.rename(columns={'res':'lgb_pred'},inplace=True)
xgb_train_pred.rename(columns={'res':'xgb_pred'},inplace=True)

In [24]:
lgb_test_pred.rename(columns={'Label':'lgb_pred'},inplace=True)
xgb_test_pred.rename(columns={'Label':'xgb_pred'},inplace=True)

In [26]:
stacking_train_data = lgb_train_pred.merge(xgb_train_pred.rename(columns={}),on='ID',how='left')


In [27]:
stacking_test_data = lgb_test_pred.merge(xgb_test_pred.rename(columns={}),on='ID',how='left')


In [28]:
train_2 = train_.merge(stacking_train_data, on='ID', how='left')
valid_2 = valid_.merge(stacking_test_data, on='ID', how='left')


In [29]:
lgb_train_pred_2, lgb_test_pred_2 = train_lgb_model(train_2[cols+['lgb_pred','xgb_pred']+['ID','Label']], 
                                                            valid_2[cols+['lgb_pred','xgb_pred']+['ID']], 
                                                            'ID', 'Label',seed=1024)

D:\anaconda3-4.3.1\envs\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


the 1 training start ...
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.13325	training's auc: 0.980293	valid_1's binary_logloss: 0.156497	valid_1's auc: 0.927022
[200]	training's binary_logloss: 0.107312	training's auc: 0.987024	valid_1's binary_logloss: 0.14203	valid_1's auc: 0.929577
Early stopping, best iteration is:
[177]	training's binary_logloss: 0.111579	training's auc: 0.985886	valid_1's binary_logloss: 0.143734	valid_1's auc: 0.929847
the 2 training start ...


D:\anaconda3-4.3.1\envs\python36\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\anaconda3-4.3.1\envs\python36\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.133661	training's auc: 0.977297	valid_1's binary_logloss: 0.156336	valid_1's auc: 0.924532
[200]	training's binary_logloss: 0.10768	training's auc: 0.988344	valid_1's binary_logloss: 0.144134	valid_1's auc: 0.927709
Early stopping, best iteration is:
[128]	training's binary_logloss: 0.124016	training's auc: 0.982637	valid_1's binary_logloss: 0.150473	valid_1's auc: 0.928151
the 3 training start ...
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.135189	training's auc: 0.974511	valid_1's binary_logloss: 0.15287	valid_1's auc: 0.935363
Early stopping, best iteration is:
[56]	training's binary_logloss: 0.158307	training's auc: 0.962656	valid_1's binary_logloss: 0.17012	valid_1's auc: 0.935549
the 4 training start ...
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.131501	training's auc: 0.97563

D:\anaconda3-4.3.1\envs\python36\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
lgb_test_pred_2.to_csv('./submit.csv',index=False)